# Dictionnaires - programmation dynamique

## Dictionnaires.

### Types muables - immuables.

In [15]:
a = [1, 2, 3]
b = a
b[0] = 4
print(a)

[4, 2, 3]


In [16]:
def f(x: int) -> int:
    x += 1
    return x
a = 0
print('a vaut : ', a)
print('f(a) renvoie : ', f(a))
print('a vaut : ', a)

a vaut :  0
f(a) renvoie :  1
a vaut :  0


In [17]:
def g(x: list) -> list:
    x.append(1)
    return x

l = [0]
print('l vaut : ', l)
print('g(l) renvoie : ', g(l))
print('l vaut : ', l)     


l vaut :  [0]
g(l) renvoie :  [0, 1]
l vaut :  [0, 1]


### Structure de dictionnaire.

In [18]:
animaux = {'chien': 'dog', 'chat': 'cat', 'oiseau': 'bird'}
pattes = {'mouche': 6, 'araignée': 8, 'mouton': 5}
cours_info = {}

In [19]:
print(animaux['chat']) # renvoie cat
pattes['mouton']=4
animaux['serpent']='snake'
cours_info[(20,9)]='Révisions'
cours_info[(11,10)]='Bases de données'
cours_info[(29,11)]='Dictionnaires'

cat


In [20]:
# Pas de cours d'informatique à Noël    
if (25, 12) in cours_info:
    del cours_info[(25, 12)]
# nombre d'animaux connus
print(len(animaux)) 

4


In [21]:
print(hash(325))
print(hash(3.14))
print(hash('Bonjour !'))
print(hash(('Charles', 3)))

325
322818021289917443
5055926124171269620
1697530509636471186


## Programmation dynamique.

### Mémoïsation - coefficients binomiaux.

Méthode récursive.

In [22]:
def binomial(n: int, p: int) -> int:
    if p == 0:
        return 1
    elif p > n:
        return 0
    else:
        return binomial(n-1, p-1)+binomial(n-1, p)


# C'est très long (plusieurs dizaines de secondes)
print(binomial(30, 20))


30045015


Avec une variable globale.

In [23]:
mem = {}


def binomial(n: int, p: int) -> int:
    if (n, p) not in mem:
        if p == 0:
            mem[(n, p)] = 1
        elif p > n:
            mem[(n, p)] = 0
        else:
            mem[(n, p)] = binomial(n-1, p-1)+binomial(n-1, p)
    return mem[(n, p)]


print(binomial(30, 20))


30045015


Avec une variable locale et une fonction imbriquée.

In [24]:
def binomial(n: int, p: int) -> int:
    mem = {}

    def b(n: int, p: int) -> int:
        if (n, p) not in mem:
            if p == 0:
                mem[(n, p)] = 1
            elif p > n:
                mem[(n, p)] = 0
            else:
                mem[(n, p)] = b(n-1, p-1)+b(n-1, p)
        return mem[(n, p)]
    return b(n, p)


print(binomial(30, 20))


30045015


Avec un paramètre par défaut. Un peu subtil !

In [25]:
def binomial(n: int, p: int, mem={}) -> int:
    if (n, p) not in mem:
        if p == 0:
            mem[(n, p)] = 1
        elif p > n:
            mem[(n, p)] = 0
        else:
            mem[(n, p)] = binomial(n-1, p-1)+binomial(n-1, p)
    return mem[(n, p)]


print(binomial(30, 20))


30045015


Calcul de bas en haut.

In [26]:
import numpy as np

def binomial(n: int) -> int:
    B = np.zeros((n+1, n+1), dtype=int)
    for n in range(n+1):
        B[n, 0] = 1
        # on affiche ligne par ligne
        ligne = str(B[n, 0])+' '
        for p in range(1, n+1):
            B[n, p] = B[n-1, p-1]+B[n-1, p]
            ligne += str(B[n, p])+' '
        print(ligne)
binomial(10)

1 
1 1 
1 2 1 
1 3 3 1 
1 4 6 4 1 
1 5 10 10 5 1 
1 6 15 20 15 6 1 
1 7 21 35 35 21 7 1 
1 8 28 56 70 56 28 8 1 
1 9 36 84 126 126 84 36 9 1 
1 10 45 120 210 252 210 120 45 10 1 


### Rendu de monnaie.

In [27]:
from math import floor
# pièces et billets en euros
euros = [1, 2, 5, 10, 20, 50]
# ancien système britannique
livres = [1, 3, 6, 12, 24, 30]


def nb(s: int, pieces: int) -> int:
    mem = {0: 0}
    # valeur signifiant que le rendu est impossible
    plafond = floor(s/min(pieces))+1

    def nb_rec(x):
        if x in mem:
            r = mem[x]
        else:
            if x < min(pieces):
                r = plafond
            else:
                sous_problemes = [
                    nb_rec(x-p) for p in pieces if p <= x]
                r = 1 + min(sous_problemes)
            mem[x] = r
        return r

    n = nb_rec(s)
    if n < plafond:
        return n
    else:
        return -1 # aucune solution


print(nb(48, euros))
print(nb(48, livres))


5
2


### Solution optimale.

In [28]:
from math import floor
euros = [1, 2, 5, 10, 20, 50]
livres = [1, 3, 6, 12, 24, 30]


def nb(s: int, pieces: list) -> int:
    mem = {0: 0}
    plafond = floor(s/min(pieces))+1

    def nb_rec(x):
        if x in mem:
            r = mem[x]
        else:
            if x < min(pieces):
                r = plafond
            else:
                sous_problemes = [
                    nb_rec(x-p) for p in pieces if p <= x]
                r = 1 + min(sous_problemes)
            mem[x] = r
        return r

    n = nb_rec(s)
    if n < plafond:
        # Recherche d'une solution optimale
        # avec les valeurs déjà calculées
        solution = []
        while s > 0:
            minimum = plafond
            piece_optimale = 0
            # on retrouve la pièce optimale pour s
            for p in pieces:
                if p <= s and nb_rec(s-p) < minimum:
                    minimum = nb_rec(s-p)
                    piece_optimale = p
            # on note la pièce optimale
            solution.append(piece_optimale)
            # on continue avec le sous-problème optimal
            s -= piece_optimale
        return n, solution
    else:
        return -1  # aucune solution


print(nb(48, euros))
print(nb(48, livres))


(5, [1, 2, 5, 20, 20])
(2, [24, 24])


## Plus longue sous-suite commune.

In [29]:
def plsc(a: str, b: str) -> int:
    mem = {(0, 0): 0} # pour la mémoïsation

    def l(i: int, j: int) -> int:
        if (i, j) not in mem:      # valeur non mémoïsée
            if i == 0 or j == 0:   # une sous-chaîne est vide
                retour = 0
            elif a[i-1] == b[j-1]: # derniers caractères égaux
                retour = 1 + l(i-1, j-1)
            else:
                retour = max(l(i, j-1), l(i-1, j))
            mem[(i, j)] = retour
        return mem[(i, j)]
    return l(len(a), len(b))


print(plsc('PROGRAMME', 'ORAGE'))

4


### Solution optimale.

In [30]:
import numpy as np

def plsc(a, b):
    mem = {(0, 0): 0}

    def l(i, j):
        if (i, j) in mem:
            retour = mem[(i, j)]
        else:
            if i == 0 or j == 0:
                retour = 0
            elif a[i-1] == b[j-1]:
                retour = 1 + l(i-1, j-1)
            else:
                retour = max(l(i, j-1), l(i-1, j))
            mem[(i, j)] = retour
        return retour
    # Recherche d'une solution optimale à partir 
    # du tableau des valeurs l(i,j) calculées
    sol = ''
    i = len(a)
    j = len(b)
    while l(i,j)>0:
        # On monte tant que l reste constant
        while l(i, j) == l(i-1, j):
            i -= 1
        # On se déplace à gauche tant que l reste constant    
        while l(i, j) == l(i, j-1):
            j -= 1
        # On a trouvé une lettre
        sol += a[i-1]
        i -= 1
        j -= 1
    # Matrice des valeurs de l(i,j)
    A = np.array([[l(i, j) for j in range(len(b)+1)]
                  for i in range(len(a)+1)])
    print(A)
    # sol[::-1] inverse l'ordre des lettres
    return l(len(a), len(b)), sol[::-1]

print(plsc('ATATACAGGTCA', 'GACTACACGACT'))
# print(plsc('PROGRAMME', 'ORAGE'))

[[0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 1 1 1 1 1 1 1]
 [0 0 1 1 2 2 2 2 2 2 2 2 2]
 [0 0 1 1 2 3 3 3 3 3 3 3 3]
 [0 0 1 1 2 3 3 3 3 3 3 3 4]
 [0 0 1 1 2 3 3 4 4 4 4 4 4]
 [0 0 1 2 2 3 4 4 5 5 5 5 5]
 [0 0 1 2 2 3 4 5 5 5 6 6 6]
 [0 1 1 2 2 3 4 5 5 6 6 6 6]
 [0 1 1 2 2 3 4 5 5 6 6 6 6]
 [0 1 1 2 3 3 4 5 5 6 6 6 7]
 [0 1 1 2 3 3 4 5 6 6 6 7 7]
 [0 1 2 2 3 4 4 5 6 6 7 7 7]]
(7, 'ATAACAT')
